# Imports

In [1]:
import sys
import os
from google.colab import drive
from google.colab import files
from dotenv import load_dotenv
import json
import warnings
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
%matplotlib inline

# Bootstrap

In [3]:
np.random.seed(31071967)

# Find and load the .env file from the current or parent directories
load_dotenv()

drive.mount('/content/drive')

with open(f"{os.getenv('PROJECT_PATH')}/src/config.json", 'r') as f:
    project_config = json.load(f)
    project_config.pop('_comment', None)
    project_config.pop('_note', None)
    f.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Download TKL data from YF

In [4]:
tickers_yf = {
    "y"     : f"{project_config['TKL']}",
 }

desired_order = [
    "Date",
    "y",
  ]

In [6]:
import yfinance as yf
import pandas as pd
from pandas_datareader import data as pdr

from datetime import date, timedelta
end_date = date.today() - timedelta(days=1)
start_date = end_date - timedelta(days=int(project_config["HISTORY_DEPTH"]))

if project_config['TKL'] == 'TNYA':
  start_date = pd.to_datetime("30.07.2022", format="%d.%m.%Y")

# ---- DOWNLOAD FROM YAHOO FINANCE ----
ts_yf = yf.download(
    tickers=list(tickers_yf.values()),
    start=start_date,
    end=end_date,
    auto_adjust=True
)["Close"]

# rename columns to readable names
rename_map = {v: k for k, v in tickers_yf.items()}
ts_yf = ts_yf.rename(columns=rename_map)

# Fill missing daily values for macro data (monthly)
ts_yf = ts_yf.fillna(method='ffill').fillna(method='bfill')
ts_yf = ts_yf.reset_index().rename(columns={"Date": "Date",})

print(f"\n\nDataset for y={project_config['TKL']}")
display(ts_yf.head(1))
display(ts_yf.tail(1))
ts_yf.info()

df = ts_yf.copy()

[*********************100%***********************]  1 of 1 completed



Dataset for y=NVDA


Ticker,Date,y
0,2006-10-23,0.48256


Ticker,Date,y
4820,2025-12-19,180.990005


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4821 entries, 0 to 4820
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    4821 non-null   datetime64[ns]
 1   y       4821 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 75.5 KB


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import random # Needed for shuffling

# Setup Directories
OUT_ROOT_DIR = f"{os.getenv('PROJECT_PATH')}{project_config['images_directory']}{project_config['TKL']}_graphs/"
PRED_IMAGE_DIR = f"{OUT_ROOT_DIR}pred/"

# If the directory exists, delete it and all its contents
#if os.path.exists(OUT_ROOT_DIR): shutil.rmtree(OUT_ROOT_DIR)
# Now create it fresh
os.makedirs(OUT_ROOT_DIR, exist_ok=True)
os.makedirs(PRED_IMAGE_DIR, exist_ok=True)

# -----------------------
# Prepare data
# -----------------------
df = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)
graph_df = df[-int(project_config['FASTAI.WINDOW_DAYS']):] # capture the last 260 days from today
last_date_obj = graph_df.iloc[-1]['Date'].date()

 # Plotting
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(graph_df['y'].values)
ax.set_axis_off()
plt.tight_layout()

image_path =f"{PRED_IMAGE_DIR}{project_config['TKL']}_{last_date_obj}_inference.png"
plt.savefig(image_path, bbox_inches="tight", pad_inches=0)
plt.close()

In [14]:
image_path

'/content/drive/MyDrive/Projects/GitHub/Stocks/images/NVDA_graphs/pred/NVDA_2025-12-19_inference.png'

In [22]:
from fastai.vision.all import *

model_path = '/content/drive/MyDrive/Projects/GitHub/Stocks/pickles/NVDA.model.fastai.pkl'

# No need to define the DataBlock or architecture again
learn_inf = load_learner(model_path)

# Run prediction
pred_class, pred_idx, probs = learn_inf.predict(image_path)

# Get the filename from image_path to extract the date
# Example: 'NVDA_2025-12-19_inference.png'
filename_from_image_path = image_path.split('/')[-1]

# Split by underscore to get the date part
# parts will be ['NVDA', '2025-12-19', 'inference.png']
parts_from_image_path = filename_from_image_path.split('_')

# The date string is at index 1
end_date_str = parts_from_image_path[1]


print(f"----- RECOMMANDATION FOR {project_config['TKL']}. HIRIZON: ONE MONTH. START DATE: {end_date_str} -----")
#print(f"File: {target_img.name}")
#print("--- PROBABILITIES ---")
# dls.vocab contains the list of classes (BUY, KEEP, etc.)
# probs contains the raw decimals

# Fix: Access vocab from learn_inf.dls
probs_map = dict(zip(learn_inf.dls.vocab, map(float, probs)))
sorted_probs = sorted(probs_map.items(), key=lambda x: x[1], reverse=True)

for category, prob in sorted_probs:
  print(f"{category}: {prob*100:.2f}%")

print(f"Final Decision: {pred_class}\n")

----- RECOMMANDATION FOR NVDA. HIRIZON: ONE MONTH. START DATE: 2025-12-19 -----
KEEP: 80.16%
BUY: 19.84%
Final Decision: KEEP



In [20]:
parts

['/content/drive/MyDrive/Projects/GitHub/Stocks/pickles/NVDA.model.fastai.pkl']